<h2 align="center"> Deploy Models with TensorFlow Serving and Docker</h2>

### Load and Preprocess Data

In [ ]:
#%%writefile -a train.py
import os
import time
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
#Souce: https://www.kaggle.com/snap/amazon-fine-food-reviews/data
!head -n 2 train.csv

Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
184502,B001BCVY4W,A1JMR1N9NBYJ1X,Mad Ethyl Flint,0,0,4,1228176000,Doesn't look like catfood!,"When you first open the can, it looks like something you would eat.  And no catfood smell! Nice sized chunks of chicken and vegetables in a lot of gravy.<br /><br />That being said, Ms Casiopia lapped up all the gravy and left the rest.  This however is not the product's fault as she has done this before with other catfoods<br /><br />I would have given it 5 stars, but since I won't be purchasing it, I gave it 4.  If your cat will eat chunks and vegetables, this product is for you.<br /><br />I have donated the remainder of the package to a less fortunate friend.<br /><br />Thank you."


In [ ]:
#%%writefile -a train.py
def load_dataset(file_path,num_sample):
  df=pd.read_csv(file_path,usecols=[6,9],nrows=num_sample)
  df.columns=['rating','title']

  text=df['title'].tolist()
  text=[str(t).encode('ascii','replace') for t in text]
  text=np.array(text,dtype=object)

  labels=df['rating'].tolist()
  labels=[1 if i>=4 else 0 if i==3 else -1 for i in labels]
  labels=np.array(pd.get_dummies(labels),dtype=int)

  return labels,text

In [ ]:
tmp_label,tmp_text= load_dataset('train.csv',100)
tmp_label.shape

(100, 3)

### Task 3: Build the Classification Model using TF Hub

In [ ]:
#%%writefile -a train.py

## https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1
## https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1



In [ ]:
def get_model():
  pretrained_model = 'https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1'
  hub_layer = hub.KerasLayer(pretrained_model, output_shape=[50], input_shape = [], dtype=tf.string, trainable=False)
  model=tf.keras.Sequential()
  model.add(hub_layer)
  model.add(tf.keras.layers.Dense(16,activation="relu"))
  model.add(tf.keras.layers.Dense(3,activation="softmax"))
  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
  model.summary()
  return model


###  Define Training Procedure

In [ ]:
#%%writefile -a train.py
def train(EPOCHS=5, BATCH_SIZE=32, TRAIN_FILE='train.csv', VAL_FILE='test.csv'):
  WORKING_DIR= os.getcwd()
  print("Loading training/validation data ...")
  y_train,x_train=load_dataset(TRAIN_FILE,num_sample=100000)
  y_val,x_val=load_dataset(VAL_FILE,num_sample=10000)

  print("Training the model ...")
  model=get_model()
  model.fit(x_train,y_train,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_data=(x_val,y_val),
          verbose=1,
          callbacks=[tf.keras.callbacks.ModelCheckpoint(os.path.join(WORKING_DIR),'model_checkpoint',
                                                        
                                                        verbose=1,
                                                        save_best_only=True,
                                                        save_weights_only=False,
                                                        mode='auto')])
  return model



###  Train and Export Model as Protobuf

In [ ]:
#%%writefile -a train.py
def export_model(model,base_path='amazon_review/'):
  path=os.path.join(base_path,str(int(time.time())))
  tf.saved_model.save(model,path)

if __name__=='__main__':
  model=train()
  export_model(model)


Loading training/validation data ...
Training the model ...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 48,191,467
Trainable params: 867
Non-trainable params: 48,190,600
_________________________________________________________________
Epoch 1/5
247/247 [==============================] - 5s 9ms/step - loss: 0.7533 - accuracy: 0.7490 - val_loss: 0.6561 - val_accuracy: 0.7838


Epoch 2/5
247/247 [==============================] - 2s 8ms/step - loss: 0.6377 - accuracy: 0.7787 - val_loss: 0.6144 - val_accuracy: 0.7838


Epoch 3/5
247/247 [==============================] - 2s 10ms/step - loss: 0.6059 - accuracy: 0.7787 - val_loss: 0.6002 - val_accuracy: 0.7837


Epoch 4/5
247/247 [==============================] - 2s 8ms/step - loss: 0.5925 - accuracy: 0.7787 - val_loss: 0.5882 - val_accuracy: 0.7846


Epoch 5/5
247/247 [==============================] - 2s 8ms/step - loss: 0.5843 - accuracy: 0.7815 - val_loss: 0.5848 - val_accuracy: 0.7861


INFO:tensorflow:Assets written to: amazon_review/1629286458/assets


INFO:tensorflow:Assets written to: amazon_review/1629286458/assets


### Test Model

#### Negative Review:

In [ ]:
test_sentence="Horrible movie"
model.predict([test_sentence])

array([[0.16203047, 0.03660675, 0.8013628 ]], dtype=float32)

#### Positive Review:

In [ ]:
test_sentence="Great movie"
model.predict([test_sentence])

array([[0.03136075, 0.01278753, 0.9558517 ]], dtype=float32)

### TensorFlow Serving with Docker

`docker pull tensorflow/serving`

`docker run -p 8500:8500 \
            -p 8501:8501 \
            --mount type=bind,\
            source=amazon_review/,\
            target=/models/amazon_review \
            -e MODEL_NAME=amazon_review \
            -t tensorflow/serving`

### Setup a REST Client to Perform Model Predictions

In [ ]:
%%writefile tf_serving_rest_client.py
import json
import requests
import sys

def get_rest_url(model_name, host='127.0.0.1', port='8501', verb='predict', version=None):
    """ generate the URL path"""
    url = "http://{host}:{port}/v1/models/{model_name}".format(host=host, port=port, model_name=model_name)
    if version:
        url += 'versions/{version}'.format(version=version)
    url += ':{verb}'.format(verb=verb)
    return url


def get_model_prediction(model_input, model_name='amazon_review', signature_name='serving_default'):
    """ no error handling at all, just poc"""

    url = get_rest_url(model_name)
    #In the row format, inputs are keyed to instances key in the JSON request.
    #When there is only one named input, specify the value of instances key to be the value of the input:
    data = {"instances": [model_input]}
    
    rv = requests.post(url, data=json.dumps(data))
    if rv.status_code != requests.codes.ok:
        rv.raise_for_status()
    
    return rv.json()['predictions']

if __name__ == '__main__':

    print("\nGenerate REST url ...")
    url = get_rest_url(model_name='amazon_review')
    print(url)
    
    while True:
        print("\nEnter an Amazon review [:q for Quit]")
        if sys.version_info[0] <= 3:
            sentence = input()
        if sentence == ':q':
            break
        model_input = sentence
        model_prediction = get_model_prediction(model_input)
        print("The model predicted ...")
        print(model_prediction)

### Setup a gRPC Client to Perform Model Predictions

In [ ]:
%%writefile tf_serving_grpc_client.py
import sys
import grpc
from grpc.beta import implementations
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2, get_model_metadata_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc


def get_stub(host='127.0.0.1', port='8500'):
    channel = grpc.insecure_channel('127.0.0.1:8500') 
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
    return stub


def get_model_prediction(model_input, stub, model_name='amazon_review', signature_name='serving_default'):
    """ no error handling at all, just poc"""
    request = predict_pb2.PredictRequest()
    request.model_spec.name = model_name
    request.model_spec.signature_name = signature_name
    request.inputs['input_input'].CopyFrom(tf.make_tensor_proto(model_input))
    response = stub.Predict.future(request, 5.0)  # 5 seconds
    return response.result().outputs["output"].float_val


def get_model_version(model_name, stub):
    request = get_model_metadata_pb2.GetModelMetadataRequest()
    request.model_spec.name = 'amazon_review'
    request.metadata_field.append("signature_def")
    response = stub.GetModelMetadata(request, 10)
    # signature of loaded model is available here: response.metadata['signature_def']
    return response.model_spec.version.value

if __name__ == '__main__':
    print("\nCreate RPC connection ...")
    stub = get_stub()
    while True:
        print("\nEnter an Amazon review [:q for Quit]")
        if sys.version_info[0] <= 3:
            sentence = raw_input() if sys.version_info[0] < 3 else input()
        if sentence == ':q':
            break
        model_input = [sentence]
        model_prediction = get_model_prediction(model_input, stub)
        print("The model predicted ...")
        print(model_prediction)